In [1]:
# Dependencies and Setup
import os
import csv
import math
import statistics
import pandas as pd
import numpy as np

In [2]:
# File to Load
school_data_row = "Resources/schools_complete.csv"
student_data_row = "Resources/students_complete.csv"

In [3]:
# Read School and Student Data File and store into Pandas Data Frames
school_df = pd.read_csv(school_data_row)

# Read School and Student Data File and store into Pandas Data Frames
student_df = pd.read_csv(student_data_row)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_df, school_df, how="left", on=["school_name", "school_name"])

# See as a DataFrame and check it out
data_df = pd.DataFrame(school_data_complete)
data_df.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


In [4]:
# Calculate the required variables
pass_math_df = data_df[data_df["math_score"]>=70]
pass_math_df["Student ID"].nunique()
percentage_pass_math = (pass_math_df["Student ID"].nunique() / data_df["Student ID"].nunique())*100
pass_reading_df = data_df[data_df["reading_score"]>=70]
pass_reading_df["Student ID"].nunique()
percentage_pass_reading = (pass_reading_df["Student ID"].nunique() / data_df["Student ID"].nunique())*100
percentage_overall_passing = (percentage_pass_math + percentage_pass_reading)/2
school_df["budget"].sum()

# Creating a summary DataFrame using the values found
district_summary_df = pd.DataFrame({
    "Total Schools":[data_df["school_name"].nunique()],
    "Total Students" : [data_df["Student ID"].nunique()],
    "Total Budget" : [school_df["budget"].sum()],
    "Average Math Score" : [data_df["math_score"].mean()],
    "Average Reading Score" : [data_df["reading_score"].mean()],
    "% Passing Math":[percentage_pass_math],
    "% Passing Reading": [percentage_pass_reading],
    "% Overall Passing Rate":[percentage_overall_passing]
})

district_summary_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,39170,24649428,78.985371,81.87784,74.980853,85.805463,80.393158


In [5]:
# Task 2. First, check the kind of variables
data_df.dtypes

Student ID        int64
student_name     object
gender           object
grade            object
school_name      object
reading_score     int64
math_score        int64
School ID         int64
type             object
size              int64
budget            int64
dtype: object

In [6]:
# Using GroupBy in order to organize the data into fields according to "school" values
grouped_schools_df = data_df.groupby(["school_name","type"])

# In order to be visualized, a data function must be used
#grouped_schools_df.count().head()

# The numeric variables can be now calculated per school 
# Total Students
total_students = grouped_schools_df["Student ID"].count()

# Budget per school
budget_per_school = grouped_schools_df["budget"].max()

# Per student budget 
budget_per_student = budget_per_school/total_students

# Average Math Score
school_average_math = grouped_schools_df["math_score"].mean()

# Average Reading Score
school_average_reading = grouped_schools_df["reading_score"].mean()

# Looking for the ones that pass math per school
pass_math_per_school = pass_math_df.groupby(["school_name"])

# In order to get the percentage passing math
perc_pass_math = (pass_math_per_school["Student ID"].count() / grouped_schools_df["Student ID"].count())*100

# Looking for the ones that pass reading per school
pass_reading_per_school = pass_reading_df.groupby(["school_name"])

# In order to get the percentage passing reading
perc_pass_reading = (pass_reading_per_school["Student ID"].count()/grouped_schools_df["Student ID"].count())*100

# Overall Passing Rate (Average of math and reading)
perc_overall_passing = (perc_pass_math+perc_pass_reading)/2

# Create a school summary
school_summary = pd.DataFrame({"Total Students" : total_students, 
                               "Total School Budget" : budget_per_school,
                               "Per Student Budget" : budget_per_student,
                               "Average Math Score" : school_average_math,
                               "Average Reading Score" : school_average_reading,
                               "% Passing Math" : perc_pass_math,
                               "% Passing Reading" : perc_pass_reading,
                               "Overall Passing Rate" : perc_overall_passing})

# Print School Summary
school_summary

,,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
school_name,type,,,,,,,,
Bailey High School,District,4976,3124928,628.0,77.048432,81.033963,66.680064,81.933280,74.306672
Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,94.133477,97.039828,95.586652
Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,65.988471,80.739234,73.363852
Ford High School,District,2739,1763916,644.0,77.102592,80.746258,68.309602,79.299014,73.804308
Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,93.392371,97.138965,95.265668
Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,66.752967,80.862999,73.807983
Holden High School,Charter,427,248087,581.0,83.803279,83.814988,92.505855,96.252927,94.379391
Huang High School,District,2917,1910635,655.0,76.629414,81.182722,65.683922,81.316421,73.500171
Johnson High School,District,4761,3094650,650.0,77.072464,80.966394,66.057551,81.222432,73.639992


In [7]:
# Sorting the "School Summary" DataFrame based on the "Overall Passing Rate" column
# Please note that this will sort from lowest to highest if no other parameter is passed
# To sort from highest to lowest, ascending=False must be passed in
sorted_school_summary = school_summary.sort_values("Overall Passing Rate", ascending=False)

# Task 3. Top performing schools (by passing rate)
sorted_school_summary.head(5)

,,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
school_name,type,,,,,,,,
Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,94.133477,97.039828,95.586652
Thomas High School,Charter,1635,1043130,638.0,83.418349,83.848930,93.272171,97.308869,95.290520
Pena High School,Charter,962,585858,609.0,83.839917,84.044699,94.594595,95.945946,95.270270
Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,93.392371,97.138965,95.265668
Wilson High School,Charter,2283,1319574,578.0,83.274201,83.989488,93.867718,96.539641,95.203679


In [8]:
# Task 4. Bottom performing schools (by passing rate)
sorted_school_summary.tail(5)

,,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
school_name,type,,,,,,,,
Ford High School,District,2739,1763916,644.0,77.102592,80.746258,68.309602,79.299014,73.804308
Johnson High School,District,4761,3094650,650.0,77.072464,80.966394,66.057551,81.222432,73.639992
Huang High School,District,2917,1910635,655.0,76.629414,81.182722,65.683922,81.316421,73.500171
Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,65.988471,80.739234,73.363852
Rodriguez High School,District,3999,2547363,637.0,76.842711,80.744686,66.366592,80.220055,73.293323


In [10]:
# Task 5. Math Scores by Grade
# Create a table that lists the average Math Score for students of each grade level 
# (9th, 10th, 11th, 12th) at each school.

# Using GroupBy in order to organize the data into fields according to "school" values
grades_per_school = data_df.groupby(["school_name", "grade"])

# Average Math Score
gps_average_math = grades_per_school["math_score"].mean()

# Average Reading Score
gps_average_reading = grades_per_school["reading_score"].mean()

# Create a school summary
gps_summary = pd.DataFrame({"Average Math Score per grade" : gps_average_math, "Average Reading Score per grade" : gps_average_reading})

# Print School Summary
#gps_summary

# Task 5. Using Pivot_table 
gps_math = pd.pivot_table(gps_summary,values="Average Math Score per grade", index="school_name", columns="grade")
gps_math.head(15)

grade,10th,11th,12th,9th
school_name,,,,
Bailey High School,76.996772,77.515588,76.492218,77.083676
Cabrera High School,83.154506,82.765560,83.277487,83.094697
Figueroa High School,76.539974,76.884344,77.151369,76.403037
Ford High School,77.672316,76.918058,76.179963,77.361345
Griffin High School,84.229064,83.842105,83.356164,82.044010
Hernandez High School,77.337408,77.136029,77.186567,77.438495
Holden High School,83.429825,85.000000,82.855422,83.787402
Huang High School,75.908735,76.446602,77.225641,77.027251
Johnson High School,76.691117,77.491653,76.863248,77.187857


In [11]:
# Task 6. Reading Scores by Grade
# Create a table that lists the average Reading Score for students of each grade level 
# (9th, 10th, 11th, 12th) at each school.
# Task 6. Using Pivot_table 
gps_reading = pd.pivot_table(gps_summary,values="Average Reading Score per grade", index="school_name", columns="grade")
gps_reading.head(15)

grade,10th,11th,12th,9th
school_name,,,,
Bailey High School,80.907183,80.945643,80.912451,81.303155
Cabrera High School,84.253219,83.788382,84.287958,83.676136
Figueroa High School,81.408912,80.640339,81.384863,81.198598
Ford High School,81.262712,80.403642,80.662338,80.632653
Griffin High School,83.706897,84.288089,84.013699,83.369193
Hernandez High School,80.660147,81.396140,80.857143,80.866860
Holden High School,83.324561,83.815534,84.698795,83.677165
Huang High School,81.512386,81.417476,80.305983,81.290284
Johnson High School,80.773431,80.616027,81.227564,81.260714


In [ ]:
# Using Pivot_table 
#gps_math = pd.pivot_table(gps_summary,values="Average Math Score per grade", index="school_name", columns="grade")
#gps_math.head()

In [ ]:
# Task 5 and 6, Phase 2
# Reset index
#gps_summary.reset_index()
#gps_summary.head()

In [ ]:
# Task 5. With Youtube help
#act_task = pd.DataFrame()
#
#for name, group in data_df.groupby("school_name"):
#    if act_task : group.set_index("grade")[["math_score"]].rename(columns={"math_score":name})
#    else:
#        end.task
#        act_task.head()

In [ ]:
# Task 5 and 6 together
# Using GroupBy in order to organize the data into fields according to "school" and "grade" values
#school_grade_groupby = data_df.groupby(["school_name","grade"])

# In order to be visualized, a data function must be used
#school_grade_groupby.count().head()

In [ ]:
# Task 5. Math Scores by Grade
# Create a table that lists the average Math Score for students of each grade level 
# (9th, 10th, 11th, 12th) at each school.

In [ ]:
# Task 6. Reading Scores by Grade
# Create a table that lists the average Reading Score for students of each grade level 
# (9th, 10th, 11th, 12th) at each school.

In [ ]:
# Task 7. Scores by School Spending
# Create a table that breaks down school performances based on average Spending Ranges (Per Student). 
# Use 4 reasonable bins to group school spending. Include in the table each of the following:
# Average Math Score
# Average Reading Score
# % Passing Math
# % Passing Reading
# Overall Passing Rate (Average of the above two)

In [ ]:
# Task 8. Scores by School Size
# Repeat the above breakdown, but this time group schools based 
# on a reasonable approximation of school size (Small, Medium, Large) 

In [ ]:
# Task 9. Scores by School Type
# Repeat the above breakdown, but this time group schools based on school type (Charter vs. District).

In [ ]:
# Task 10. You must include a written description of at least two observable trends based on the data.